In [1]:
import sys

def sum(v1,v2):
    result = int(v1) + int(v2)
    print(result)


def main(argv):
    sum(argv[1], argv[2])

if __name__ == "__main__":
    main(sys.argv)


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cx_Oracle
import os
import pandas as pd

LOCATION = r"C:\oraclexe"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록

In [3]:
dsn=cx_Oracle.makedsn('localhost',1521,'xe')
db=cx_Oracle.connect('hr','a1234')

cursor=db.cursor()
#cursor.execute("select * from ugotfilm_movie_choice")
#for i in cursor:
#    print(i)

In [4]:
df=pd.read_sql('select usercode, moviecode, count(*) as count from ugotfilm_movie_choice group by usercode, moviecode order by usercode', con=db)

C:\Users\ez210416\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
df

,USERCODE,MOVIECODE,COUNT
0,1,201334,5
1,1,315162,7
2,1,505642,21
3,1,646389,1
4,1,746524,5
5,1,1013870,20
6,2,505642,1
7,3,201334,3
8,3,505642,2
9,3,536554,1


In [6]:
user= 2

In [7]:
recommendation_pivot = df.pivot(index='USERCODE', columns='MOVIECODE', values='COUNT')

In [8]:
recommendation_pivot.fillna(0, inplace=True)

In [9]:
recommendation_pivot

MOVIECODE,201334,315162,505642,536554,646389,653851,676547,738005,746524,949484,985939,1013870,1058949
USERCODE,,,,,,,,,,,,,
1,5.0,7.0,21.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,20.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,10.0,2.0,0.0,0.0,0.0,8.0,0.0,0.0,23.0,0.0,8.0,21.0
9,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def movie_seen(user_id):
  return recommendation_pivot.loc[user_id][recommendation_pivot.loc[user_id]>0]

In [11]:
movie_seen(user)

MOVIECODE
505642    1.0
Name: 2, dtype: float64

In [12]:
small_test = recommendation_pivot.T

In [13]:
small_test

USERCODE,1,2,3,4,5,9,10
MOVIECODE,,,,,,,
201334,5.0,0.0,3.0,0.0,0.0,0.0,0.0
315162,7.0,0.0,0.0,0.0,10.0,0.0,0.0
505642,21.0,1.0,2.0,2.0,2.0,2.0,1.0
536554,0.0,0.0,1.0,0.0,0.0,0.0,0.0
646389,1.0,0.0,0.0,0.0,0.0,0.0,1.0
653851,0.0,0.0,1.0,0.0,0.0,0.0,0.0
676547,0.0,0.0,0.0,0.0,8.0,0.0,0.0
738005,0.0,0.0,0.0,0.0,0.0,0.0,1.0
746524,5.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
small_test_corr = small_test.corr()
small_test_corr

USERCODE,1,2,3,4,5,9,10
USERCODE,,,,,,,
1,1.000000,0.660343,0.290164,0.527083,-0.074479,0.527083,0.212722
2,0.660343,1.000000,0.453928,0.887244,-0.129950,0.887244,0.527046
3,0.290164,0.453928,1.000000,0.342840,-0.366075,0.342840,0.075550
4,0.527083,0.887244,0.342840,1.000000,-0.214466,1.000000,0.414758
5,-0.074479,-0.129950,-0.366075,-0.214466,1.000000,-0.214466,-0.339470
9,0.527083,0.887244,0.342840,1.000000,-0.214466,1.000000,0.414758
10,0.212722,0.527046,0.075550,0.414758,-0.339470,0.414758,1.000000


In [15]:
small_test_corr[user].sort_values(ascending=False)[1:11]

USERCODE
4     0.887244
9     0.887244
1     0.660343
10    0.527046
3     0.453928
5    -0.129950
Name: 2, dtype: float64

In [16]:
def nearest_user(user_id, n):
  return small_test_corr.loc[user_id].sort_values(ascending=False)[1:n+1]

In [17]:
res = nearest_user(user,1)

In [18]:
res.index[0]

4

In [19]:
movie_seen(res.index[0])

MOVIECODE
505642    2.0
985939    1.0
Name: 4, dtype: float64

In [20]:
movie_seen(user)

MOVIECODE
505642    1.0
Name: 2, dtype: float64

In [21]:
unseen = set(movie_seen(res.index[0]).index) - set(movie_seen(user).index)


In [22]:
print(unseen)

{985939}
